In [17]:
# 用户输入句子
inputStr = input("请输入句子：")
# 自动去掉结尾多余空格
inputStr = inputStr.strip() # 去掉首尾空格
def text_to_bmes(input_text, output_file):
    with open(output_file, 'w', encoding='utf-8') as f:
        for char in input_text:
            if char == '\n':
                f.write('\n')
            else:
                f.write(f'{char} O\n')
        f.write('\n')  #保证bmes文件里面的空行


input_text = inputStr

output_file = "./ResumeNER/predict.char.bmes"

text_to_bmes(input_text, output_file)




In [18]:
import pickle
import sys
sys.path.append("..")  # 将上级目录添加到Python路径
from utils import load_model, extend_maps, prepocess_data_for_lstmcrf
from data import build_corpus,build_map
from evaluating import Metrics
from evaluate import ensemble_evaluate

from os.path import join
from codecs import open


In [19]:
def build_corpus(split, make_vocab=True, data_dir="./ResumeNER"):
    """读取数据"""
    print(f"Provided split value: {split}")  # 添加输出以查看传入的split值
    assert split in ['train', 'dev', 'test','predict']

    word_lists = []
    tag_lists = []
    with open(join(data_dir, split+".char.bmes"), 'r', encoding='utf-8') as f:
        word_list = []
        tag_list = []
        for line in f:
            # if line != '\r\n':
            #     word, tag = line.strip('\n').split()
            #     word_list.append(word)
            #     tag_list.append(tag)
            if line.strip():  # 如果不是空行
                word, tag = line.strip().split()
                word_list.append(word)
                tag_list.append(tag)
            else:
                word_lists.append(word_list)
                tag_lists.append(tag_list)
                word_list = []
                tag_list = []

    # 如果make_vocab为True，还需要返回word2id和tag2id
    if make_vocab:
        word2id = build_map(word_lists)
        tag2id = build_map(tag_lists)
        return word_lists, tag_lists, word2id, tag2id
    else:
        return word_lists, tag_lists

In [20]:
predict_word_lists, predict_tag_lists = build_corpus("predict", make_vocab=False,data_dir="./ResumeNER")

Provided split value: predict


In [21]:
train_word_lists, train_tag_lists, word2id, tag2id = build_corpus("train",data_dir="./ResumeNER")

Provided split value: train


In [22]:
crf_word2id, crf_tag2id = extend_maps(word2id, tag2id, for_crf=True)

In [23]:
bilstm_model = load_model('./ckpts/bilstm_crf.pkl')
bilstm_model.model.bilstm.bilstm.flatten_parameters()  # remove warning

In [24]:
predict_word_lists, predict_tag_lists = prepocess_data_for_lstmcrf(
        predict_word_lists, predict_tag_lists, test=True
    )


In [25]:
lstmcrf_pred, target_tag_list = bilstm_model.test(predict_word_lists, predict_tag_lists,
                                                  crf_word2id, crf_tag2id)

In [26]:
lstmcrf_pred

[['B-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'M-ORG',
  'E-ORG']]